In [1]:
import time
import numpy as np

import _init_paths
from libs.scheduler import PfScheduler
from libs.models.naive_solver import NaiveURLLCSolver
from libs.models.greedy_solver import GreedyURLLCSolver
from utils.generator import generate
from utils.metrics import get_embb_utility
# from utils.visualization import draw_matrix

/Users/yinhao/Documents/Code/Joint-eMBB-URLLC
/Users/yinhao/Documents/Code/Joint-eMBB-URLLC/libs/models


In [2]:
# parameter setting
rb_size = 15
rb_num = 254
embb_num = 3
embb_slot_len = 7
urllc_num = 6
urllc_slot_len = 1
max_sim_time_slot_len = embb_slot_len

embb_rb_req = [100, 50, 120]
embb_rb_size = [60, 120, 30]
urllc_rb_req = [10, 10, 10, 10, 10, 10]
urllc_rb_size = [30, 30, 30, 30, 30, 30]
urllc_slot_start = [1, 2, 3, 4, 5, 6]

In [3]:
#urllc scheduler setting
URLLCSolver = NaiveURLLCSolver
# URLLCSolver = GreedyURLLCSolver

# generate
embb_users, urllc_users, RB_map = generate(rb_size,
    rb_num,
    embb_num, 
    embb_slot_len, 
    urllc_num,
    urllc_slot_len,
    embb_rb_req,
    embb_rb_size,
    urllc_rb_req,
    urllc_rb_size,
    urllc_slot_start,
    latency=1,
    error_rate=1e-5,
    mcs_error=1e-5,
)


In [4]:
urllc_users[0].__dict__.items()

dict_items([('user_info', {'id': 4}), ('rb_size', 30), ('rb_num_req', 10), ('rb_num_ass', 0), ('rb_start', 0), ('sche_times', 0), ('active', 1), ('slot_len', 1), ('slot_start', 1), ('latency', 1), ('delay', 0), ('miss', 0), ('ori_embb', []), ('error_rate', 1e-05), ('mcs_error', 1e-05), ('retrans', 0)])

In [5]:
global_time = 0
global_timeout_urllc_users = []

# embb_time_slot pfs scheduler
pf_scheduler = PfScheduler(RB_map, embb_users)
pf_scheduler.allocate_resource()


    
# urllc_time_slot get urllc_active_come_list, setting urllc come from global_time=1 and later
urllc_users.sort(key=lambda x:x.slot_start)
urllc_come_time = np.array([u.slot_start for u in urllc_users])

global_time = 0
global_timeout_urllc_users = []

# embb_time_slot pfs scheduler
pf_scheduler = PfScheduler(RB_map, embb_users)
pf_scheduler.allocate_resource()

In [6]:
embb_users[2].__dict__.items()

dict_items([('user_info', {'id': 3}), ('rb_size', 30), ('rb_num_req', 120), ('rb_num_ass', 120), ('rb_start', 0), ('sche_times', 2), ('active', 1), ('slot_len', 7), ('DRC', 0.5142857142857143), ('rate_slot', 0.0), ('rate_avg', 0.48000000000000004), ('replace_num', 0)])

In [7]:
# urllc_time_slot get urllc_active_come_list, setting urllc come from global_time=1 and later
urllc_users.sort(key=lambda x:x.slot_start)
urllc_come_time = np.array([u.slot_start for u in urllc_users])

delay_users = []
timer = []
# urllc scheduler loop for all the urllc_time_slot without considering embb user rescheduling
while(global_time < max_sim_time_slot_len):
    global_time += 1
    indexes = np.where(urllc_come_time==global_time)[0]
    if len(indexes) == 0:
        continue
    urllc_users_list = delay_users + urllc_users.copy()[indexes[0]:indexes[-1]+1]

    
    urllc_scheduler = URLLCSolver(RB_map, embb_users, urllc_users_list)
    
    start = time.time()
    ass_users, delay_users, timeout_users = urllc_scheduler.allocate_resource()
    timer.append(time.time() - start)

    global_timeout_urllc_users.append(timeout_users)
    
    print(RB_map.__dict__.items())
    # urllc leave
    urllc_scheduler.leave(ass_users)
    
# get_embb_utility miss_list and time cost for all the urllc scheduler
embb_utility = get_embb_utility(embb_users)
total_time_cost = sum(timer)

# visualize (TODO)
for user in embb_users:
    print(user .__dict__.items())
print("\n")
print(RB_map.__dict__.items())

# next embb_time_slot (not show here)


dict_items([('rb_avi', 254), ('rb_size', 30), ('bitmap', array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])), ('border_bitmap_id', 3)])
dict_items([('rb_avi', 254), ('rb_size', 30), ('bitmap', array([5, 5

In [8]:
urllc_come_time

array([1, 2, 3, 4, 5, 6])